# CS4287 Assignment 2: Deep Reinforcement Learning (Atari)

**Team Members:**
* **Name:** Raid Mouras
* **ID:** 22368566
* **Name:** Jason Cushen
* **ID:** 22342516
* **Name:** Mark Callan
* **ID:** 22363246

In [10]:
%pip install gymnasium[atari] gymnasium[accept-rom-license] torch torchvision numpy opencv-python matplotlib
!AutoROM --accept-license

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
'AutoROM' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import deque
import cv2
import time

# --- 1. CONFIGURATION (Breakout) ---
ENV_NAME = "BreakoutNoFrameskip-v4"
GAMMA = 0.99                # Discount factor
BATCH_SIZE = 32             # Batch size
LR = 1e-4                   # Learning Rate
EPSILON_START = 1.0         # 100% random actions
EPSILON_FINAL = 0.02        # 2% random actions
EPSILON_DECAY = 150000      # Exploration frames
REPLAY_SIZE = 100000        # Memory size
TARGET_UPDATE = 1000        # Network update frequency

# --- 2. HARDWARE CHECK ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Setup Complete for: {ENV_NAME}")
print(f"Target Device: {device}")

if device.type == 'cuda':
    print(f"✅ GPU DETECTED: {torch.cuda.get_device_name(0)}")
else:
    print("❌ WARNING: CPU DETECTED.")

Setup Complete for: BreakoutNoFrameskip-v4
Target Device: cuda
✅ GPU DETECTED: NVIDIA RTX 1000 Ada Generation Laptop GPU


In [12]:
# --- CELL 3: PREPROCESSING (The Eyes) ---

class AtariWrapper(gym.Wrapper):
    def __init__(self, env, k=4):
        super().__init__(env)
        self.k = k  # k=4 means we stack 4 frames together to see "motion"
        self.frames = deque([], maxlen=k)
        
        # We define what the AI sees: A stack of 4 images, each 84x84 pixels
        self.observation_space = gym.spaces.Box(
            low=0, high=255, shape=(k, 84, 84), dtype=np.uint8
        )

    def reset(self, **kwargs):
        # Reset the game and fill the stack with the first frame 4 times
        obs, info = self.env.reset(**kwargs)
        processed_frame = self._process(obs)
        for _ in range(self.k):
            self.frames.append(processed_frame)
        return self._get_obs(), info

    def step(self, action):
        # Play one step, capture the new frame, process it, and add to stack
        obs, reward, terminated, truncated, info = self.env.step(action)
        self.frames.append(self._process(obs))
        return self._get_obs(), reward, terminated, truncated, info

    def _process(self, frame):
        # 1. Turn color into Grayscale (Black & White)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        # 2. Shrink to 84x84 pixels (Less data for GPU to crunch)
        frame = cv2.resize(frame, (84, 84), interpolation=cv2.INTER_AREA)
        return frame

    def _get_obs(self):
        # Stack frames into a numpy array (4, 84, 84)
        return np.array(self.frames)

print("✅ Cell 3 Complete: Preprocessing wrapper defined.")

✅ Cell 3 Complete: Preprocessing wrapper defined.


In [13]:
# --- CELL 4: THE NETWORK (The Brain) ---

class DQN(nn.Module):
    def __init__(self, input_shape, num_actions):
        super(DQN, self).__init__()
        
        # 1. Convolutional Layers (Visual Cortex)
        # These layers "scan" the 84x84 images to find patterns (balls, paddles).
        self.features = nn.Sequential(
            # Conv 1: Scans with an 8x8 filter. Captures big objects.
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            
            # Conv 2: Scans with a 4x4 filter. Captures smaller details.
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            
            # Conv 3: Scans with a 3x3 filter. Captures fine interactions.
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )
        
        # 2. Fully Connected Layers (Decision Making)
        # These layers take the patterns found above and decide "Left" or "Right".
        self.fc = nn.Sequential(
            nn.Linear(64 * 7 * 7, 512), # 64 filters * 7x7 size = 3136 inputs
            nn.ReLU(),
            nn.Linear(512, num_actions) # Output: One score for every button
        )

    def forward(self, x):
        # Pass input through the visual layers
        x = self.features(x)
        # Flatten: Squash the 3D maps into a 1D vector so the linear layers can read it
        x = x.view(x.size(0), -1)
        # Pass through decision layers to get Q-values
        return self.fc(x)

print("✅ Cell 4 Complete: Neural Network defined.")

✅ Cell 4 Complete: Neural Network defined.


In [8]:
# --- CELL 5: THE TRAINING LOOP ---

# 1. Initialize Everything
env = gym.make(ENV_NAME)
env = AtariWrapper(env) # Apply our "Eyes"
agent_net = DQN(env.observation_space.shape, env.action_space.n).to(device) # The Actor
target_net = DQN(env.observation_space.shape, env.action_space.n).to(device) # The Reference
target_net.load_state_dict(agent_net.state_dict()) # Sync them up
optimizer = optim.Adam(agent_net.parameters(), lr=LR)
replay_buffer = deque(maxlen=REPLAY_SIZE)

steps = 0
rewards_history = []

print(f"🚀 TRAINING STARTED on {device}...")
print("The agent will play randomly for the first ~20 mins to fill memory.")

# 2. The Loop (We run for 1000 episodes for the start)
for episode in range(1000): 
    state, _ = env.reset()
    total_reward = 0
    done = False
    
    while not done:
        steps += 1
        
        # --- A. SELECT ACTION (Epsilon Greedy) ---
        # Calculate how "random" we should be right now
        epsilon = EPSILON_FINAL + (EPSILON_START - EPSILON_FINAL) * np.exp(-1. * steps / EPSILON_DECAY)
        
        if random.random() > epsilon:
            # Smart Move: Ask the AI "What is the best move?"
            with torch.no_grad():
                state_t = torch.tensor(state, device=device, dtype=torch.float32).unsqueeze(0) / 255.0
                action = agent_net(state_t).argmax().item()
        else:
            # Random Move: Explore the game
            action = env.action_space.sample()
            
        # --- B. PLAY THE STEP ---
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        
        # --- C. SAVE TO MEMORY ---
        replay_buffer.append((state, action, reward, next_state, done))
        state = next_state
        total_reward += reward
        
        # --- D. TRAIN (The "Q-Learning Update" - 4 Marks) ---
        if len(replay_buffer) > BATCH_SIZE:
            # 1. Grab a random batch of memories
            batch = random.sample(replay_buffer, BATCH_SIZE)
            states, actions, rewards, next_states, dones = zip(*batch)
            
            # 2. Convert to GPU Tensors
            states = torch.tensor(np.array(states), device=device, dtype=torch.float32) / 255.0
            actions = torch.tensor(actions, device=device, dtype=torch.int64).unsqueeze(1)
            rewards = torch.tensor(rewards, device=device, dtype=torch.float32).unsqueeze(1)
            next_states = torch.tensor(np.array(next_states), device=device, dtype=torch.float32) / 255.0
            dones = torch.tensor(dones, device=device, dtype=torch.float32).unsqueeze(1)
            
            # 3. Calculate "Current Q" (What we thought would happen)
            curr_q = agent_net(states).gather(1, actions)
            
            # 4. Calculate "Target Q" (What actually happened + future guess)
            with torch.no_grad():
                max_next_q = target_net(next_states).max(1)[0].unsqueeze(1)
                target_q = rewards + (GAMMA * max_next_q * (1 - dones))
                
            # 5. Calculate Error and Update Weights
            loss = nn.SmoothL1Loss()(curr_q, target_q)
            optimizer.zero_grad()
            loss.backward() # <--- This is the actual learning moment
            optimizer.step()
            
        # --- E. UPDATE TARGET NETWORK ---
        if steps % TARGET_UPDATE == 0:
            target_net.load_state_dict(agent_net.state_dict())

    rewards_history.append(total_reward)
    
    # Print progress every 10 episodes
    if episode % 10 == 0:
        avg_score = np.mean(rewards_history[-100:])
        print(f"Episode: {episode} | Score: {total_reward} | Avg Score: {avg_score:.2f} | Steps: {steps}")

env.close()
print("Training Finished.")

NameNotFound: Environment `BreakoutNoFrameskip` doesn't exist.